In [2]:
from dotenv import load_dotenv
import os
from google import genai
from google.genai import types as gt
import requests

In [3]:
# 读取 .env
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
JIRA_TOKEN = os.getenv("JIRA_TOKEN")
JIRA_USER = os.getenv("JIRA_USER")
JIRA_HOST = os.getenv("JIRA_HOST")
PROJECT_KEY = os.getenv("PROJECT_KEY")

# 初始化 Gemini Client
client = genai.Client(api_key=GEMINI_API_KEY)

In [4]:
# 获取最近更新的 Jira 任务
def jira_get(path, params=None):
    url = f"{JIRA_HOST}{path}"
    r = requests.get(url, params=params, auth=(JIRA_USER, JIRA_TOKEN))
    r.raise_for_status()
    return r.json()

In [5]:
# 1. 从 Jira 拉取任务
issues = jira_get("/rest/api/3/search", params={
    "jql": f"project={PROJECT_KEY} ORDER BY updated DESC",
    "maxResults": 10,
    "fields": "summary,status,assignee,duedate,priority,created,updated"
})["issues"]

In [6]:
# 2. 格式化任务列表为字符串
def format_issues_to_str(issues):
    """
    将 Jira issues 转换成字符串，便于交给 AI 分析
    """
    lines = []
    for i in issues:
        f = i["fields"]

        key = i.get("key", "N/A")
        summary = f.get("summary", "无标题")
        status = f.get("status", {}).get("name", "未知状态")
        assignee = (f.get("assignee") or {}).get("displayName", "未分配")
        duedate = f.get("duedate", "无截止日期")
        priority = (f.get("priority") or {}).get("name", "无优先级")
        created = f.get("created", "未知时间")
        updated = f.get("updated", "未知时间")
        labels = ", ".join(f.get("labels", [])) or "无标签"
        parent = (f.get("parent") or {}).get("key", "无父任务")

        lines.append(
            f"{key} | {summary} | 状态: {status} | 优先级: {priority} | 负责人: {assignee} | 截止: {duedate} | 创建: {created} | 更新: {updated} | 标签: {labels} | 父任务: {parent}"
        )

    return "\n".join(lines)
issues_str = format_issues_to_str(issues)
print("Formatted issues:")
print(issues_str)

Formatted issues:
BTS-6 | （示例）修复数据库连接错误 | 状态: To Do | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.696+0900 | 更新: 2025-09-19T15:52:57.103+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-5 | （示例）解决 API 超时的问题 | 状态: In Progress | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.419+0900 | 更新: 2025-09-19T12:49:41.485+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-4 | （示例）加快下拉菜单响应速度 | 状态: In Progress | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.148+0900 | 更新: 2025-09-19T12:49:41.343+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-3 | （示例）修复按钮对齐的问题 | 状态: In Review | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:39.840+0900 | 更新: 2025-09-19T12:49:41.198+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-2 | （示例）后端缺陷 | 状态: In Progress | 优先级: Medium | 负责人: 未分配 | 截止: 2025-10-03 | 创建: 2025-09-19T12:49:38.930+0900 | 更新: 2025-09-19T12:49:39.505+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-1 | （示例）用户界面缺陷 | 状态: In Progress | 优先级: Medium | 负责人: 未分配 | 截止: 2025-09-26 | 创建: 2025-09-19T12:49:37.665+0900 | 更新: 2025-09-19T12:49:39.301+0900 | 标签

In [7]:
# 3. 调用 Gemini 做分析
prompt = (
    "你是资深项目管理顾问，请根据以下 Jira 任务列表，分析风险并提出对策：\n\n"
    + issues_str
)

print("\n=== Prompt to Gemini ===")
print(prompt)


=== Prompt to Gemini ===
你是资深项目管理顾问，请根据以下 Jira 任务列表，分析风险并提出对策：

BTS-6 | （示例）修复数据库连接错误 | 状态: To Do | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.696+0900 | 更新: 2025-09-19T15:52:57.103+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-5 | （示例）解决 API 超时的问题 | 状态: In Progress | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.419+0900 | 更新: 2025-09-19T12:49:41.485+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-4 | （示例）加快下拉菜单响应速度 | 状态: In Progress | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.148+0900 | 更新: 2025-09-19T12:49:41.343+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-3 | （示例）修复按钮对齐的问题 | 状态: In Review | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:39.840+0900 | 更新: 2025-09-19T12:49:41.198+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-2 | （示例）后端缺陷 | 状态: In Progress | 优先级: Medium | 负责人: 未分配 | 截止: 2025-10-03 | 创建: 2025-09-19T12:49:38.930+0900 | 更新: 2025-09-19T12:49:39.505+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-1 | （示例）用户界面缺陷 | 状态: In Progress | 优先级: Medium | 负责人: 未分配 | 截止: 2025-09-26 | 创建: 2025-09-19T12:49:37.6

In [8]:
resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=gt.GenerateContentConfig(temperature=0.3)
)

In [ ]:
# 4. 获取 Gemini 分析结果
gemini_response = resp.text
# print("=== AI 分析结果 ===")
# print(gemini_response)

=== AI 分析结果 ===
您好！作为资深项目管理顾问，我仔细审阅了您提供的Jira任务列表。根据这些信息，我将为您分析当前项目面临的风险，并提出相应的对策。

---

### **项目管理风险分析报告**

**整体评估：**
根据提供的Jira任务列表，当前项目管理状态存在**严重且系统性的风险**。最核心的问题是**所有任务均未分配负责人，且绝大部分任务缺乏明确的优先级**。这表明项目在基础的执行和控制层面存在根本性缺陷，若不立即纠正，将极大地影响项目进度、质量和最终交付。

**主要风险点：**

1.  **责任分配与所有权缺失 (高风险)**
    *   **风险描述：** 所有任务（BTS-1至BTS-6）的“负责人”字段均为“未分配”。这意味着没有任何人对这些任务负有明确的责任。
    *   **潜在影响：**
        *   **任务无人认领：** 实际工作可能根本没有开始，或正在停滞。
        *   **责任不明确：** 无法追踪谁应该完成什么，导致问责制缺失。
        *   **沟通障碍：** 团队成员不知道就特定任务应与谁沟通，导致协作效率低下。
        *   **进度停滞：** 即使任务状态显示“In Progress”或“In Review”，但没有负责人，其真实进展存疑，极有可能导致进度停滞。
        *   **BTS-3 (In Review) 风险：** 处于“In Review”状态但无负责人，意味着没有人会进行代码审查或功能验证，可能导致缺陷遗漏或质量问题。

2.  **优先级混乱与目标不明确 (高风险)**
    *   **风险描述：** 绝大多数任务的“优先级”字段为“无优先级”。仅有BTS-1和BTS-2被标记为“Medium”。
    *   **潜在影响：**
        *   **工作方向迷失：** 团队成员不清楚哪些任务更重要，应优先处理，可能导致资源浪费在低价值任务上。
        *   **关键问题延迟：** 重要的缺陷（如数据库连接错误、API超时）可能无法得到及时解决，影响系统稳定性或用户体验。
        *   **无法有效决策：** 在资源有限的情况下，项目经理无法基于优先级进行有效决策和资源调配。

3.  **进

In [ ]:
# 5. 定义将结果返回 Jira 的函数
def jira_post(path, data):
    """通用 POST 函数"""
    url = f"{JIRA_HOST}{path}"
    r = requests.post(url, json=data, auth=(JIRA_USER, JIRA_TOKEN))
    r.raise_for_status()
    return r.json()

# 6. 执行添加评论的操作
def add_comment_to_jira(issue_key, comment_text):
    """将评论添加到指定的 Jira issue"""
    comment_endpoint = f"/rest/api/3/issue/{issue_key}/comment"
    
    # Jira API v3 需要使用 Atlassian Document Format
    comment_data = {
        "body": {
            "type": "doc",
            "version": 1,
            "content": [
                {
                    "type": "paragraph",
                    "content": [
                        {
                            "type": "text",
                            "text": comment_text
                        }
                    ]
                }
            ]
        }
    }
    
    try:
        result = jira_post(comment_endpoint, comment_data)
        print(f"成功将评论添加到 issue {issue_key}。")
        return result
    except requests.exceptions.HTTPError as e:
        print(f"添加到 issue {issue_key} 失败: {e}")
        print(f"Response body: {e.response.text}")
        return None

In [ ]:
# 将下面的 'BTS-6' 替换为您希望评论的真实 JIRA ISSUE KEY !!!
issue_to_comment = "BTS-6"
add_comment_to_jira(issue_to_comment, gemini_response)

成功将评论添加到 issue BTS-6。


{'self': 'https://xuxiang.atlassian.net/rest/api/3/issue/10005/comment/10002',
 'id': '10002',
 'author': {'self': 'https://xuxiang.atlassian.net/rest/api/3/user?accountId=712020%3A3b2a0587-3cc4-4074-b281-6f334e79410c',
  'accountId': '712020:3b2a0587-3cc4-4074-b281-6f334e79410c',
  'emailAddress': 'xuxiang5012@gmail.com',
  'avatarUrls': {'48x48': 'https://secure.gravatar.com/avatar/d05cc9f1554abc7adee496310d9c4252?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FX-2.png',
   '24x24': 'https://secure.gravatar.com/avatar/d05cc9f1554abc7adee496310d9c4252?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FX-2.png',
   '16x16': 'https://secure.gravatar.com/avatar/d05cc9f1554abc7adee496310d9c4252?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FX-2.png',
   '32x32': 'https://secure.gravatar.com/avatar/d05cc9f1554abc7adee496310d9c4252?d=https%3A%2F%2Favatar-management-